In [1]:
import os
import torch
from torch import nn
import random
from PIL import Image
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import ImageFile

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
class MultiFolderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = []

        # Enumerate subdirectories and assign a unique label to each
        for label, subdir in enumerate(sorted(os.listdir(root_dir))):
            subdir_path = os.path.join(root_dir, subdir)
            if os.path.isdir(subdir_path):
                self.class_names.append(subdir)
                for filename in os.listdir(subdir_path):
                    file_path = os.path.join(subdir_path, filename)
                    self.images.append(file_path)
                    self.labels.append(label)  # Assign label based on folder index

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        
        try:
            # Open the image and convert to RGB
            image = Image.open(img_path).convert('RGB')
        except OSError as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a blank image or handle it in another way (e.g., skip this image)
            return self.__getitem__((idx + 1) % len(self.images))

        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [5]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
root_dir = 'images/'

In [7]:
dataset = MultiFolderDataset(root_dir, transform=data_transforms)

In [8]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [9]:
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Validation Dataset Size: {len(val_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")

Train Dataset Size: 11116
Validation Dataset Size: 2382
Test Dataset Size: 2383


In [12]:
print(f"Class Names: {dataset.class_names}")

Class Names: ['body', 'body-art', 'face', 'face-art', 'hand', 'hand-art']


In [13]:
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features

/home/tugberk/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tugberk/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
model.fc = nn.Linear(num_features, len(dataset.class_names))

In [15]:
model = model.to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
import copy


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)

        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        # Deep copy the model
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    print(f'Best val Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model

In [18]:
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 0/9
----------
Train Loss: 0.4611 Acc: 0.8308
Val Loss: 0.4093 Acc: 0.8623
Epoch 1/9
----------
Train Loss: 0.3118 Acc: 0.8894
Val Loss: 0.2885 Acc: 0.8929
Epoch 2/9
----------
Train Loss: 0.2271 Acc: 0.9169
Val Loss: 0.2732 Acc: 0.9072
Epoch 3/9
----------
Train Loss: 0.2092 Acc: 0.9245
Val Loss: 0.3096 Acc: 0.8854
Epoch 4/9
----------
Train Loss: 0.1758 Acc: 0.9362
Val Loss: 0.5705 Acc: 0.8249
Epoch 5/9
----------
Train Loss: 0.1519 Acc: 0.9441
Val Loss: 0.1891 Acc: 0.9303
Epoch 6/9
----------
Train Loss: 0.1241 Acc: 0.9536
Val Loss: 0.2168 Acc: 0.9307
Epoch 7/9
----------
Train Loss: 0.1111 Acc: 0.9608
Val Loss: 0.2349 Acc: 0.9207
Epoch 8/9
----------
Train Loss: 0.1092 Acc: 0.9601
Val Loss: 0.2872 Acc: 0.9156
Epoch 9/9
----------
Train Loss: 0.0993 Acc: 0.9654
Val Loss: 0.2236 Acc: 0.9236
Best val Acc: 0.9307


In [19]:
from sklearn.metrics import classification_report, confusion_matrix


def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(classification_report(all_labels, all_preds, target_names=dataset.class_names))
    print(confusion_matrix(all_labels, all_preds))


In [20]:
evaluate_model(trained_model, test_loader)

              precision    recall  f1-score   support

        body       0.90      0.94      0.92       431
    body-art       0.94      0.91      0.93       416
        face       0.97      0.89      0.93       464
    face-art       0.93      0.98      0.95       570
        hand       0.89      0.91      0.90       177
    hand-art       0.93      0.92      0.92       325

    accuracy                           0.93      2383
   macro avg       0.93      0.93      0.93      2383
weighted avg       0.93      0.93      0.93      2383

[[407  20   2   1   1   0]
 [ 30 379   1   0   0   6]
 [ 10   0 414  40   0   0]
 [  1   0   8 557   2   2]
 [  0   0   0   0 161  16]
 [  4   4   1   0  16 300]]
